In [1]:
#### generate(topic) > evaluate > not good    > optimize
#                               > good enough > approved

In [2]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Literal, Annotated
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage
import operator

In [3]:
class TweetEvaluation(BaseModel):
    evaluation: Literal['approved', 'needs_improvement'] = Field(description="Final Feedback Evaluation Result")
    feedback: str = Field(description='feedback description of the tweet')


In [4]:
generator_llm = ChatOpenAI(model = 'gpt-4o-mini')
evaluator_llm = ChatOpenAI(model = 'gpt-4o-mini')
structured_eveluator_llm = evaluator_llm.with_structured_output(TweetEvaluation)
optimizer_llm = ChatOpenAI(model = 'gpt-4o-mini')

In [5]:
# state

class TweetState(TypedDict):

    topic: str
    tweet: str
    evaluation: Literal['approved', 'needs_improvement']
    feedback: str
    iteration: int
    max_iterations: int

    tweet_history: Annotated[list[str], operator.add]
    feedback_history: Annotated[list[str], operator.add]

In [6]:
# task for nodes

def generate_tweet(state: TweetState) -> TweetState:
    # prompt
    messages = [
                SystemMessage(content="You are funny and clever Twitter influencer."),
                HumanMessage(content=f"""
                Write a short, original, and hilarious on the topic: "{state['topic']}"
                    Rules :
                        - Do NOT use question-answer format.
                        - Max 280 characters.
                        - Use observational humor, irony, sarcasm, or cultural references.
                        - Think in meme logic, punchlines, or relatable takes.
                        - Use simple, day to day english
                        - This is version {state['iteration'] + 1}.""")
                ]

    # tweet generator model
    response = generator_llm.invoke(messages).content

    return {'tweet': response, 'tweet_history':[response]}
    # return response

In [7]:
def evaluate_tweet(state: TweetState) -> TweetState:
    # prompt
    messages = [
                SystemMessage(content="You are a ruthless, no-laugh given twitter critic. You evaluate Tweets based on humor, originality, virality and tweet format."), 
                HumanMessage(content=f"""
                Evaluate the following tweet:

                Tweet : "{state['tweet']}"
                Use the criteria below to evaluate the tweet:
                1. Originality — Is this fresh, or have you seen it a hundred times before?
                2. Humor - Did it genuinely make you smile, laugh, or chuckle?
                3. Punchiness — Is it short, sharp, and scroll-stopping?
                4. Virality Potential Would people retweet or share it?
                5. Format — Is it a well-formed tweet (not a setup-punchline joke, not a Q&A joke, and under 280 characters)?

                Auto-reject if:
                - It's written in question-answer format (e.g., or "What happens when...", "what happends when...")
                - It exceeds 280 characters
                - It reads like a traditional setup-punchline joke
                - Dont end with generic, throwaway, or deflating lines that weaken the humor (e.g., "universe" or vague summaries)

                Respond ONLY in structured format:
                - evaluation: "approved" or "needs_improvement"
                - feedback:   One paragraph explaining the strengths and weaknesses""")
                ]
    # evaluator model(needs structures ouput) - 
    response = structured_eveluator_llm.invoke(messages)

    return {'evaluation':response.evaluation, 'feedback': response.feedback, 'feedback_history':[response.feedback]}

    #

In [8]:
def optimize_tweet(state: TweetState) -> TweetState:
    # prompt

    messages = [
        SystemMessage(content="You punch up tweets for virality and humor based on given feedback"),
        HumanMessage(content=f"""
                            Improve the folowing tweet based on this feedback:
                            "{state['feedback']}"

                            Topic:  "{state['topic']}"
                            original_tweet: 
                            {state['tweet']}
                            re-write it as a short viral worthy tweet.
                            avoid Q&A style and below 280 characters
                            """)
    ]

    response = optimizer_llm.invoke(messages).content
    iteration = state['iteration'] + 1

    
    return {'tweet' : response, 'iteration': iteration, 'tweet_history':[response]}

In [9]:
def route_evaluation(state: TweetState) -> Literal['approved', 'needs_improvement']:

    if state['evaluation'] == 'approved' or state['iteration'] >= state['max_iterations']:
        return 'approved'
    else:
        return 'needs_improvement'


In [10]:
#  graph
graph = StateGraph(TweetState)

graph.add_node('generate_tweet', generate_tweet) 
graph.add_node('evaluate_tweet', evaluate_tweet) 
graph.add_node('optimize_tweet', optimize_tweet)

In [11]:
# edges

graph.add_edge(START, 'generate_tweet')
graph.add_edge('generate_tweet', 'evaluate_tweet')
graph.add_conditional_edges('evaluate_tweet', route_evaluation, {'approved': END, 'needs_improvement': 'optimize_tweet'})

graph.add_edge('optimize_tweet', 'evaluate_tweet')

workflow = graph.compile()

In [12]:
workflow.invoke({'topic':'Air India', 'iteration':1, 'max_iterations':5})

{'topic': 'Air India',
 'tweet': "Air India: where flight attendants deserve medals for patience and your luggage is basically on a world tour. Four hours waiting for your bag? Sounds like it's having a better vacation than you! 🧳✈️ #AirIndiaAdventures",
 'evaluation': 'approved',
 'feedback': 'This tweet successfully combines originality with humor, depicting the common frustrations of air travel in a relatable way. The concept of luggage going on a "world tour" is fresh and creatively highlights a frequent issue, adding a humorous twist that many can resonate with. It maintains a punchy format that is concise and easily digestible for scrolling, enhancing its potential for virality. Overall, this tweet is well-structured and effectively engages the audience.',
 'iteration': 3,
 'max_iterations': 5,
 'tweet_history': ['Air India: where the flight attendants have a PhD in patience and your baggage has a better social life than you. They say time flies when you’re having fun—try telling

In [14]:
workflow.invoke({'topic':'djdas', 'iteration':1, 'max_iterations':5})

{'topic': 'djdas',
 'tweet': 'When Dad DJs the Wi-Fi fix, get ready for classic jams and dropped signals. “Turn it up!” he yells from the basement as he spins the router like a record. We might not get internet, but at least we’ve got the *dad* beats! 🎧📶 #DadVibes #WiFiWars',
 'evaluation': 'approved',
 'feedback': 'This tweet showcases originality by blending the relatable scenario of dad attempting to fix Wi-Fi with a humorous twist on DJ culture, creating a vivid and funny image of him ‘spinning the router like a record.’ The humor is present and charming, evoking a smile through its playful language and the clever wordplay of ‘dad beats.’ It is punchy yet descriptive enough to capture attention, and it adheres to the 280-character limit, maintaining a well-formed tweet structure. The hashtags are relevant and enhance its potential for engagement, making it likely to be shared and retweeted.',
 'iteration': 4,
 'max_iterations': 5,
 'tweet_history': ["You ever notice how DJDAS sound